In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, PReLU, ReLU
from tensorflow.keras.regularizers import l2, l1_l2
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import he_normal
from tensorflow.keras.optimizers.schedules import ExponentialDecay

In [2]:
import tensorflow as tf

# Check TensorFlow version (should be a version that supports GPU)
print("TensorFlow version:", tf.__version__)

# Check if GPU is available
if tf.test.is_built_with_cuda():
    print("TensorFlow was built with CUDA")
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)
    else:
        print("No GPU devices available.")
else:
    print("TensorFlow was not built with CUDA")

TensorFlow version: 2.12.0
TensorFlow was not built with CUDA


In [5]:
# Load the dataset
file_path = 'train.csv'  # Replace with your CSV file path
data = pd.read_csv(file_path)

In [6]:
# Applying logarithmic transformations
data['full_sq_log'] = np.log1p(data['full_sq'])
data['leisure_count_500_log'] = np.log1p(data['leisure_count_500'])
data['life_sq_log'] = np.log1p(data['life_sq'])
data['cafe_count_500_price_high_log'] = np.log1p(data['cafe_count_500_price_high'])

In [7]:
# Add log-transformed features to the list of key features
key_features = ['full_sq', 'life_sq', 'floor', 'leisure_count_500', 'cafe_count_1000_price_high',
                'mosque_count_500', 'mosque_count_1000', 'cafe_count_500_price_high',
                'cafe_count_1000_price_high', 'cafe_count_500_price_4000', 'culture_objects_top_25_raion', 
                'leisure_count_1000', 'trc_sqm_500', 'church_count_500', 'cafe_count_1000_price_1500', 
                'mosque_count_1500', 'cafe_count_500', 'big_church_count_500',
                'public_transport_station_min_walk', 'metro_min_walk', 'kindergarten_km', 'preschool_km',
                'full_sq_log', 'life_sq_log', 'leisure_count_500_log', 'cafe_count_500_price_high_log']  # Add log-transformed features
data = data[key_features + ['price_doc']]

In [8]:
# Handling any potential infinities or NaNs
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.fillna(0, inplace=True)

# Splitting the data into features (X) and target variable (y)
X = data.drop('price_doc', axis=1)
y = data['price_doc']

# Splitting the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature Scaling using Robust Scaling
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [9]:

batch_size = 64 # Adjust the batch size
l2_reg = 0.01

In [10]:

# Configuration
initial_learning_rate = 0.003  # Initial learning rate
epochs_to_decay = 5            # Decay every 3 epochs
decay_rate = 0.9               # Reduce to 90% of the previous rate
decay_steps = 2269 * 5  # Steps after which to apply decay

# Set up the learning rate schedule
lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps=decay_steps, decay_rate=decay_rate, staircase=True)



In [16]:
model = Sequential([
    Dense(4112, kernel_initializer='he_normal', input_shape=(X_train_scaled.shape[1],)),
    PReLU(),
    BatchNormalization(),
    Dense(2056),
    PReLU(),
    BatchNormalization(),
    Dense(1028),
    PReLU(),
    BatchNormalization(),
    Dense(512, kernel_initializer='he_normal'),
    PReLU(),
    BatchNormalization(),
    Dense(256),
    PReLU(),
    BatchNormalization(),
    Dense(128, kernel_initializer='he_normal'),
    PReLU(),
    Dense(64),
    PReLU(),
    Dense(32, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4), kernel_initializer='he_normal'),
    PReLU(),
    Dense(1, activation='linear', kernel_initializer='he_normal')
])


In [17]:
model.compile(optimizer=Adam(learning_rate=lr_schedule), loss='mean_squared_error', metrics=[RootMeanSquaredError(name='rmse')])



In [13]:
# Implementing Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

In [18]:
# Training the Model with the specified batch size
history = model.fit(X_train_scaled, y_train, batch_size=128, epochs=120, validation_data=(X_val_scaled, y_val), callbacks=[early_stopping])

Epoch 1/120
  59/1135 [>.............................] - ETA: 2:09 - loss: 700386356756480.0000 - rmse: 26464814.0000

In [ ]:
# Model Summary
model.summary()

In [ ]:
# Evaluating the model
results = model.evaluate(X_val_scaled, y_val)
print(f"Validation RMSE: {results[1]}")

In [ ]:
# Load the test dataset
test_file_path = 'test.csv'  # Replace with your test CSV file path
test_data = pd.read_csv(test_file_path)

# Applying the same logarithmic transformations as in the training data
test_data['full_sq_log'] = np.log1p(test_data['full_sq'])
test_data['leisure_count_500_log'] = np.log1p(test_data['leisure_count_500'])
test_data['life_sq_log'] = np.log1p(test_data['life_sq'])
test_data['cafe_count_500_price_high_log'] = np.log1p(test_data['cafe_count_500_price_high'])

# Handling any potential infinities or NaNs
test_data.replace([np.inf, -np.inf], np.nan, inplace=True)
test_data.fillna(0, inplace=True)

# Selecting the same features as used in the training data
X_test = test_data[['full_sq', 'life_sq', 'floor', 'leisure_count_500', 'cafe_count_1000_price_high',
                    'mosque_count_500', 'mosque_count_1000', 'cafe_count_500_price_high',
                    'cafe_count_1000_price_high', 'cafe_count_500_price_4000', 'culture_objects_top_25_raion', 
                    'leisure_count_1000', 'trc_sqm_500', 'church_count_500', 'cafe_count_1000_price_1500', 
                    'mosque_count_1500', 'cafe_count_500', 'big_church_count_500',
                    'public_transport_station_min_walk', 'metro_min_walk', 'kindergarten_km', 'preschool_km',
                    'full_sq_log', 'life_sq_log', 'leisure_count_500_log', 'cafe_count_500_price_high_log']]


# Scale the test data using the same scaler as the training data
# Assuming the scaler is already fitted on the training data
X_test_scaled = scaler.transform(X_test)

# Make predictions using the trained model
# Assuming 'model' is your trained model instance from the same notebook session
predictions = model.predict(X_test_scaled)

# Prepare submission
submission = pd.DataFrame({
    'row ID': test_data['row ID'],  # Replace 'ID' with the identifier column of your test dataset
    'price_doc': predictions.flatten()
})
submission.to_csv('predictions.csv', index=False)
